In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastf1 as ff1
import os
import seaborn as sns
from fastf1.core import Laps
from fastf1 import utils
from fastf1 import plotting
from datetime import timedelta
from timple.timedelta import strftimedelta
from fastf1 import plotting


In [20]:
class F1Class:
    def __init__(self, year, place, modality):
        self.year = year
        self.place = place
        self.modality = modality
        if not os.path.exists('cache'):
            os.makedirs('cache')
        ff1.Cache.enable_cache('cache')
        self.event = ff1.get_session(self.year,self.place,self.modality)
        self.event.load()
        plotting.setup_mpl()
    
    def plot_bargraph_times(self):
        list_fastest_laps = list()    
        for drv in self.event.results['Abbreviation']:
                drvs_fastest_lap = self.event.laps.pick_driver(drv).pick_fastest()
                list_fastest_laps.append(drvs_fastest_lap)
        fastest_laps = Laps(list_fastest_laps).sort_values(by='LapTime').reset_index(drop=True)

        pole_lap = fastest_laps.pick_fastest()
        fastest_laps['LapTimeDelta'] = fastest_laps['LapTime'] - pole_lap['LapTime']

        team_colors = list()
        for index, lap in fastest_laps.iterlaps():
            color = ff1.plotting.team_color(lap['Team'])
            team_colors.append(color)


            
        fig, ax = plt.subplots(figsize=(12, 6.75))
        ax.barh(fastest_laps.index, fastest_laps['LapTimeDelta'],
                color=team_colors, edgecolor='grey')
        ax.set_yticks(fastest_laps.index)
        ax.set_yticklabels(fastest_laps['Driver'])

        # show fastest at the top
        ax.invert_yaxis()

            # draw vertical lines behind the bars
        ax.set_axisbelow(True)
        ax.xaxis.grid(True, which='major', linestyle='--', color='black', zorder=-1000)

        lap_time_string = strftimedelta(pole_lap['LapTime'], '%m:%s.%ms')

        plt.suptitle(f"{self.event.event['EventName']} {self.year} \n"
                f"Fastest Lap: {lap_time_string} ({pole_lap['Driver']})")
        
    def telemetry_between_drivers(self, drv1: str , drv2:str):
        drv1_laps = self.event.laps.pick_driver(drv1)
        drv1_telemetry = drv1_laps.pick_fastest().get_telemetry().add_distance()
        

        drv2_laps = self.event.laps.pick_driver(drv2)
        drv2_telemetry = drv2_laps.pick_fastest().get_telemetry().add_distance()

        color_drv1 = ff1.plotting.team_color(drv1_laps['Team'].reset_index(drop = True)[0])
        color_drv2 = ff1.plotting.team_color(drv2_laps['Team'].reset_index(drop = True)[0])

        if(color_drv1 == color_drv2):
             color_drv2 = "#B9DCE3"

        delta_time , ref_tel , compare_tel = utils.delta_time( drv1_laps.pick_fastest(), drv2_laps.pick_fastest())

        plot_ratios = [1, 3, 3, 2]
        

        fig, ax = plt.subplots(4, gridspec_kw={'height_ratios': plot_ratios}, figsize=(12, 6.75))
        lap_time_drv1_string = strftimedelta(drv1_laps.pick_fastest()['LapTime'], '%m:%s.%ms')
        lap_time_drv2_string = strftimedelta(drv2_laps.pick_fastest()['LapTime'], '%m:%s.%ms')

        plt.suptitle(f"{self.event.event['EventName']} {self.year} \n"
                            f"{drv1} ({lap_time_drv1_string}) vs {drv2} ({lap_time_drv2_string}) ")


        ax[0].plot(ref_tel['Distance'], delta_time, color = color_drv1)
        ax[0].axhline(0, color = color_drv2)
        ax[0].set(ylabel=f"Gap to {drv2} (s)")

        ax[1].plot(drv1_telemetry['Distance'],drv1_telemetry['Speed'], label = f'{drv1}', color = color_drv1 )
        ax[1].plot(drv2_telemetry['Distance'], drv2_telemetry['Speed'], label = f'{drv2}', color = color_drv2)
        ax[1].set(ylabel = 'Speed', xlabel = "Distance")
        ax[1].legend(loc = "lower right")

        ax[2].plot(drv1_telemetry['Distance'],drv1_telemetry['Throttle'], label = f'{drv1}', color = color_drv1)
        ax[2].plot(drv2_telemetry['Distance'], drv2_telemetry['Throttle'], label = f'{drv2}', color = color_drv2)
        ax[2].set(ylabel = 'Throttle', xlabel = "Distance")
        ax[2].legend(loc = "lower right")

        ax[3].plot(drv1_telemetry['Distance'],drv1_telemetry['Brake'], label = f'{drv1}', color = color_drv1)
        ax[3].plot(drv2_telemetry['Distance'], drv2_telemetry['Brake'], label = f'{drv2}', color = color_drv2)
        ax[3].set(ylabel = 'Brake', xlabel = "Distance")
        ax[3].legend(loc = "lower right")
    
    
    def engine_manufacter(self):
        driver_pole = self.event.laps.pick_driver(self.event.results['Abbreviation'][0]).pick_fastest()
        lap_time_pole_string = strftimedelta( driver_pole['LapTime'], '%m:%s.%ms')
        plt.rcParams["figure.figsize"] = [12, 6]
        fig, ax = plt.subplots()

        for drv in self.event.results['Abbreviation']:
                drv_fastest_lap = self.event.laps.pick_driver(drv).pick_fastest()
                deltaTime = drv_fastest_lap['LapTime'] - driver_pole['LapTime'] 
                color = ff1.plotting.team_color(drv_fastest_lap['Team'])
                ax.scatter(drv_fastest_lap.get_telemetry()['Speed'].max(), pd.Timedelta(deltaTime).total_seconds(), color = color)
                ax.text(drv_fastest_lap.get_telemetry()['Speed'].max() + 0.1, pd.Timedelta(deltaTime).total_seconds() + 0.03, drv)
        ax.set(xlabel='Speed- Telem Max. (km/h)', ylabel= 'LapTime Delta(s)')
        plt.suptitle(f"LapTime by Engine Manufacturer\n{self.event.event['EventName']} {self.year} \n"
                        f"Fastest Lap: ({lap_time_pole_string}) ({driver_pole['Driver']})")
        plt.savefig('Engine', dpi=350)

    def tyre_strategy(self):
        fig, ax = plt.subplots(figsize=(12,8))

        pitstops = self.event.laps[['LapNumber', 'Stint', 'Driver']].copy()
        pitstops['Pitstop'] = ~pitstops.Stint.eq(pitstops.Stint.shift())
        pitstops = pitstops[pitstops.Pitstop] 
        pitstops = pitstops[pitstops.LapNumber > 1]
        pitstops.plot.scatter('LapNumber', 'Driver', ax=ax, color='purple', s=100)

        df = self.event.laps[['Driver', 'LapNumber', 'Compound']]
        df = pd.merge(self.event.results[['Abbreviation']], df, left_on='Abbreviation', right_on='Driver')
        for tyre,color in zip(['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET'], ['red', 'yellow', 'white', 'green', 'blue']):
            df = self.event.laps[self.event.laps.Compound == tyre][['Driver', 'LapNumber', 'Compound']]
            df.plot.scatter('LapNumber', 'Driver', ax=ax, color=color, s=16)
        ax.invert_yaxis()
        ax.set_title('Tyre Strategy')
        plt.show()
    
    def race_trace_chart(self, drivers = []):
        average_driver_laptime = []
        for driver in self.event.results['Abbreviation'][1:10]:
            driver_laps = self.event.laps.pick_driver(driver)
            driver_laps['LapTimeSeconds'] = driver_laps['Time'].dt.total_seconds()
            average_driver_laptime.append(driver_laps['LapTimeSeconds'].reset_index(drop = True))
        virtual_driver = pd.DataFrame(average_driver_laptime)

        plt.rcParams["figure.figsize"] = [20, 8]
        plt.rcParams["figure.autolayout"] = True
        fig, ax = plt.subplots()
        color_list = []

        if(drivers == []):
           drivers = self.event.results['Abbreviation'] 
        
        for driver in drivers:
            driver_laps = self.event.laps.pick_driver(driver)
            driver_laps['LapTimeSeconds'] = driver_laps['Time'].dt.total_seconds()
            color = ff1.plotting.team_color(driver_laps['Team'].reset_index(drop = True)[0])
            if color in color_list:
                ax.plot(driver_laps['LapNumber'], virtual_driver.mean().reset_index(drop=True)[:len(driver_laps['LapTimeSeconds'])] - driver_laps['LapTimeSeconds'].reset_index(drop=True), marker = 'o', label= driver, color = color, ls='--')
            else:
                ax.plot(driver_laps['LapNumber'], virtual_driver.mean().reset_index(drop=True)[:len(driver_laps['LapTimeSeconds'])] - driver_laps['LapTimeSeconds'].reset_index(drop=True), marker = 'o', label= driver)
            color_list.append(color)

        ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
        ax.set(xlabel='Laps', ylabel= '<-- Driver behind // Driver ahead --> ')
        plt.title(f"Race Trace - {self.event.event['EventName']} {self.year}")
        plt.savefig('Gap_To_Virtual_Driver_Bahrain.png', dpi=350)

    def plot_top_speed(self):
        drslist = []
        for driver in self.event.laps.Driver.unique():
            drs = self.event.laps.pick_driver(driver).get_telemetry()[['Speed', 'DRS']].groupby('DRS').max()
            withoutDRS = drs[drs.index < 5]['Speed'].max()
            withDRS = drs[drs.index > 5]['Speed'].max()
            drslist.append({'Driver':driver, 'DRS':withDRS, 'noDRS': withoutDRS})
        topspeeds = pd.DataFrame(drslist)

        fig, ax = plt.subplots(figsize=(15,10))
        topspeeds.plot.scatter('Driver', 'DRS', ax=ax, color='orange', s=16, label='DRS')
        topspeeds.plot.scatter('Driver', 'noDRS', ax=ax, color='lightskyblue', s=16, label='no DRS')
        ax.axhline(topspeeds.DRS.mean(), color='orange' )
        ax.axhline(topspeeds.noDRS.mean(), color='lightskyblue', linestyle = '--')
        ax.set_title('Top Speed')
        ax.legend()
        plt.show()
    
    
    def plot_bargraph_team(self, session = 'q3'):
        
        q1, q2 , q3 = self.event.laps.split_qualifying_sessions()
        if(session == 'q1'):
            best_team_laptimes = q1.groupby('Team')['LapTime'].min().reset_index()
        elif(session == 'q2'):
            best_team_laptimes = q2.groupby('Team')['LapTime'].min().reset_index()
        else:
            best_team_laptimes = q3.groupby('Team')['LapTime'].min().reset_index()
        
        best_team_laptimes['LapTime'] = pd.to_timedelta(best_team_laptimes['LapTime'] - best_team_laptimes['LapTime'].min()).dt.total_seconds()
        plt.figure(figsize=(12, 6.75))
        sns.barplot(x='LapTime', y='Team', data=best_team_laptimes.sort_values(by='LapTime', ascending= True),  palette=[ff1.plotting.team_color(team) for team in best_team_laptimes.sort_values(by='LapTime', ascending= True)['Team']])
        plt.grid(axis='both', linestyle='--', alpha=0.7)
        plt.show()

    
    def plot_car_characteristics(self):
        laps = self.event.laps
        min_lap_indexes = laps.groupby('Team')['LapTime'].idxmin()
        drivers_with_fastest_lap = laps.loc[min_lap_indexes, ['Driver', 'LapTime', 'Team']]
        df = drivers_with_fastest_lap = drivers_with_fastest_lap.sort_values(by='LapTime')
        high_speed = []
        plt.rcParams["figure.figsize"] = [8, 8]
        fig, ax = plt.subplots()
        for drv in df['Driver']:
            color = ff1.plotting.team_color(laps.pick_driver(drv)['Team'].reset_index(drop = True)[0])
            telemetry = laps.pick_driver(drv).pick_fastest().get_telemetry()
            high_speed.append((telemetry['Speed'].mean(),color, telemetry['Speed'].max(), laps.pick_driver(drv)['Team'].reset_index(drop = True)[0]))

        ax.set(xlabel='Mean Speed (km/h)', ylabel= 'Top Speed (km/h)')
        plt.scatter(list(zip(*high_speed))[0],list(zip(*high_speed))[2], color = list(zip(*high_speed))[1])

        for i in range(len(high_speed)):
            ax.annotate(list(zip(*high_speed))[3][i], (list(zip(*high_speed))[0][i], list(zip(*high_speed))[2][i] + 0.3))

        plt.title("Car Characteristics")
        plt.savefig('car_characteristics', dpi=350)